In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import matplotlib.dates as mdates

pd.set_option("display.max_columns", 200)
sns.set_theme(style="whitegrid")



ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# put your raw Benin CSV path here; do NOT commit the CSV
raw_path = Path("data/benin_raw.csv")

if not raw_path.exists():
    raw_path.parent.mkdir(parents=True, exist_ok=True)
    sample_data = [
        {"Timestamp": "2023-01-01 00:00:00", "GHI": 0.0, "DNI": 0.0, "DHI": 0.0, "ModA": 0.0, "ModB": 0.0,
         "Tamb": 22.5, "RH": 65.0, "WS": 1.2, "WSgust": 2.0, "WStdev": 0.3, "WD": 45.0, "WDstd": 5.0,
         "BP": 1010.5, "TModA": 23.0, "TModB": 22.8, "Cleaning": 0, "Comments": "placeholder"},
        {"Timestamp": "2023-01-01 01:00:00", "GHI": 35.0, "DNI": 50.0, "DHI": 10.0, "ModA": 32.0, "ModB": 30.0,
         "Tamb": 22.0, "RH": 66.0, "WS": 1.5, "WSgust": 2.3, "WStdev": 0.4, "WD": 50.0, "WDstd": 6.0,
         "BP": 1010.2, "TModA": 23.1, "TModB": 23.0, "Cleaning": 0, "Comments": "placeholder"},
        {"Timestamp": "2023-01-01 02:00:00", "GHI": 120.0, "DNI": 180.0, "DHI": 25.0, "ModA": 118.0, "ModB": 115.0,
         "Tamb": 23.0, "RH": 60.0, "WS": 2.1, "WSgust": 3.2, "WStdev": 0.6, "WD": 55.0, "WDstd": 7.0,
         "BP": 1009.8, "TModA": 24.0, "TModB": 23.7, "Cleaning": 0, "Comments": "placeholder"},
        {"Timestamp": "2023-01-01 03:00:00", "GHI": 220.0, "DNI": 300.0, "DHI": 40.0, "ModA": 215.0, "ModB": 210.0,
         "Tamb": 24.5, "RH": 55.0, "WS": 3.0, "WSgust": 4.0, "WStdev": 0.8, "WD": 60.0, "WDstd": 8.0,
         "BP": 1009.0, "TModA": 25.5, "TModB": 25.0, "Cleaning": 0, "Comments": "placeholder"},
        {"Timestamp": "2023-01-01 04:00:00", "GHI": 310.0, "DNI": 420.0, "DHI": 55.0, "ModA": 305.0, "ModB": 300.0,
         "Tamb": 26.0, "RH": 50.0, "WS": 3.5, "WSgust": 4.8, "WStdev": 0.9, "WD": 65.0, "WDstd": 9.0,
         "BP": 1008.5, "TModA": 27.0, "TModB": 26.4, "Cleaning": 1, "Comments": "placeholder"},
    ]
    sample_df = pd.DataFrame(sample_data)
    sample_df.to_csv(raw_path, index=False)
    print(f"Created placeholder dataset at {raw_path}. Replace with the real Benin CSV when available.")

df = pd.read_csv(raw_path, parse_dates=["Timestamp"])
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'data/benin_raw.csv'

## Task 2 — Data Profiling, Cleaning & EDA Checklist

This notebook follows the Week 0 brief:

- Summary statistics & missing values
- Outlier detection and cleaning actions
- Time-series exploration
- Cleaning impact analysis
- Correlation & relationship analysis
- Wind, distribution, temperature analysis
- Bubble chart for GHI vs. Tamb with RH/BP context

Replace the placeholder data with the real Benin dataset (`data/benin_raw.csv`) before drawing conclusions.


In [ ]:
# Basic dataset overview
print(f"Rows: {len(df):,}")
print(f"Columns: {len(df.columns)}")
print("\nData types:")
print(df.dtypes)

summary_stats = df.describe(datetime_is_numeric=True).T
summary_stats


In [ ]:
# Missing value summary
missing_counts = df.isna().sum()
missing_ratio = missing_counts / len(df)
missing_df = (
    pd.concat([missing_counts, missing_ratio], axis=1)
      .rename(columns={0: "missing_count", 1: "missing_ratio"})
      .sort_values("missing_count", ascending=False)
)
missing_df


In [ ]:
if missing_counts.sum() > 0:
    fig, ax = plt.subplots(figsize=(8, 4))
    sns.barplot(x=missing_df.index, y="missing_ratio", data=missing_df.reset_index(), ax=ax)
    ax.set_ylabel("Missing ratio")
    ax.set_xlabel("Column")
    ax.set_title("Missing values per column")
    ax.yaxis.set_major_formatter(lambda x, pos: f"{x:.0%}")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
else:
    print("No missing values detected.")


In [ ]:
# Outlier detection using Z-scores
numeric_cols = df.select_dtypes(include=["number"]).columns
z_scores = df[numeric_cols].apply(lambda col: np.abs(stats.zscore(col, nan_policy="omit")))
outlier_flags = (z_scores > 3)
outlier_summary = outlier_flags.sum().sort_values(ascending=False)

outlier_summary


In [ ]:
potential_outliers = df[outlier_flags.any(axis=1)]
print(f"Potential outlier rows: {len(potential_outliers)}")
potential_outliers.head()


In [ ]:
# Basic cleaning steps
df_clean = df.copy()

# 1. Sort by timestamp and drop duplicated timestamps
if "Timestamp" in df_clean.columns:
    df_clean = df_clean.sort_values("Timestamp").drop_duplicates(subset=["Timestamp"], keep="first")

# 2. Fill missing numeric values with linear interpolation (if any)
numeric_cols = df_clean.select_dtypes(include=["number"]).columns
df_clean[numeric_cols] = df_clean[numeric_cols].interpolate(method="linear", limit_direction="both")

# 3. Fill categorical/string columns with forward fill
categorical_cols = df_clean.select_dtypes(include=["object"]).columns
for col in categorical_cols:
    df_clean[col] = df_clean[col].ffill().bfill()

print(f"Rows after cleaning: {len(df_clean):,}")
df_clean.head()


In [ ]:
clean_path = raw_path.with_name("benin_clean.csv")
df_clean.to_csv(clean_path, index=False)
print(f"Saved cleaned dataset to {clean_path} (git-ignored).")


In [ ]:
# Time series plots for irradiance and temperature
if "Timestamp" in df_clean.columns:
    fig, ax = plt.subplots(figsize=(10, 5))
    key_cols = [col for col in ["GHI", "DNI", "DHI", "Tamb"] if col in df_clean.columns]
    for col in key_cols:
        ax.plot(df_clean["Timestamp"], df_clean[col], label=col)
    ax.set_xlabel("Timestamp")
    ax.set_ylabel("Value")
    ax.set_title("Time series overview")
    ax.legend()
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d %H:%M"))
    fig.autofmt_xdate()
    plt.tight_layout()
else:
    print("Timestamp column missing; skipping time series plot.")


In [ ]:
# Cleaning impact: compare means before/after cleaning for key columns
impact_cols = [col for col in ["ModA", "ModB", "GHI", "DNI"] if col in df.columns]
if impact_cols:
    comparison = pd.DataFrame({
        "raw_mean": df[impact_cols].mean(),
        "clean_mean": df_clean[impact_cols].mean()
    })
    comparison["delta"] = comparison["clean_mean"] - comparison["raw_mean"]
    comparison
else:
    print("No module or irradiance columns available for impact comparison.")


In [ ]:
# Correlation heatmap on cleaned numeric data
if len(numeric_cols) > 1:
    corr = df_clean[numeric_cols].corr()
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(corr, ax=ax, cmap="coolwarm", center=0, annot=True, fmt=".2f")
    ax.set_title("Correlation heatmap (cleaned data)")
    plt.tight_layout()
else:
    print("Not enough numeric columns for correlation heatmap.")


In [ ]:
# Wind speed & direction distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
if "WS" in df_clean.columns:
    sns.histplot(df_clean["WS"], ax=axes[0], bins=10, kde=True)
    axes[0].set_title("Wind speed distribution (m/s)")
else:
    axes[0].set_visible(False)

if "WD" in df_clean.columns:
    sns.histplot(df_clean["WD"], ax=axes[1], bins=12)
    axes[1].set_title("Wind direction distribution (degrees)")
else:
    axes[1].set_visible(False)

plt.tight_layout()


In [ ]:
# Temperature relationships
if {"Tamb", "RH"}.issubset(df_clean.columns):
    fig, ax = plt.subplots(figsize=(6, 4))
    hue_param = "Cleaning" if "Cleaning" in df_clean.columns else None
    sns.scatterplot(data=df_clean, x="Tamb", y="RH", hue=hue_param, ax=ax)
    ax.set_title("Ambient temperature vs. relative humidity")
else:
    print("Tamb/RH columns missing; skipping temperature analysis.")


In [ ]:
# Bubble chart: GHI vs Tamb (bubble size = RH, color = Cleaning)
required_cols = {"GHI", "Tamb", "RH"}
if required_cols.issubset(df_clean.columns):
    fig, ax = plt.subplots(figsize=(6, 4))
    color_values = df_clean["Cleaning"] if "Cleaning" in df_clean.columns else np.zeros(len(df_clean))
    scatter = ax.scatter(
        df_clean["GHI"],
        df_clean["Tamb"],
        s=df_clean["RH"].fillna(df_clean["RH"].median()) * 2,
        c=color_values,
        cmap="viridis",
        alpha=0.7,
        edgecolor="k"
    )
    ax.set_xlabel("GHI (W/m²)")
    ax.set_ylabel("Tamb (°C)")
    ax.set_title("Bubble chart: GHI vs Tamb with RH size")
    if "Cleaning" in df_clean.columns:
        legend1 = ax.legend(*scatter.legend_elements(prop="colors"), title="Cleaning")
        ax.add_artist(legend1)
else:
    print("GHI, Tamb, RH columns required for bubble chart.")


### Next steps

- Replace the sample dataset with the real Benin CSV from Week 0 resources.
- Update the cleaning logic if new issues appear (e.g., sensor-specific anomalies, calibration offsets).
- Document key findings (peaks, correlations, anomalies) inline as Markdown.
- Promote reusable transformations to `src/` or `scripts/` for collaboration.

When satisfied, export `data/benin_clean.csv` for downstream cross-country analysis but keep it out of Git as per the brief.
